## Data Science Intern at Let's Grow more June 2023

## Intermediate Level Task

## Next Word Prediction

## Mohammad Aamir Rangrez

# **Table of Contents:**
* Import Libraries
* Getting the Data
* Data exploration/Analysis
* Spitting Dataset into onw wod each
* Making list of sorted unique words
* Features Engineering ad one hot Encoding 
* Storing features and corresponding labels
* Building the model
* Training our model
* Loadin our Model
* Making Predictions
* Function for prediction
* Result


# Importing Libraries

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from nltk.tokenize import RegexpTokenizer
from keras.models import Sequential, load_model
from keras.layers import LSTM
from keras.layers.core import Dense,Activation
from tensorflow.keras.optimizers import RMSprop
import pickle
import heapq

# Uploading Dataset from system

In [2]:
from ipywidgets import FileUpload
from IPython.display import display
upload = FileUpload(accept='.txt',multiple=True)
display(upload)

FileUpload(value={}, accept='.txt', description='Upload', multiple=True)

FileUpload(value={}, accept='.txt', description='Upload', multiple=True)

In [3]:
text = open('1999.txt',encoding='UTF-8').read().lower()
print('orpus lenght:',len(text))

orpus lenght: 581888
orpus lenght: 581888


# Spitting Dataset into one word each

In [4]:
tokenizer = RegexpTokenizer(r'\w+')
words = tokenizer.tokenize(text)                           

# Making list of sorted unique words

In [6]:
unique_word  = np.unique(words)
unique_word_index = dict((c,i) for i,c in enumerate(unique_word))

# Features Engineering and one Hot Encoding

In [8]:
word_length = 5
prev_words = []
next_words = []
for i in range(len(words) - word_length):
    prev_words.append(words[i:i + word_length])
    next_words.append(words[i + word_length])
print(prev_words[0])
print(next_words[0])

['project', 'gutenberg', 's', 'the', 'adventures']
of
['project', 'gutenberg', 's', 'the', 'adventures']
of


# Storing features and corresponding labels

In [10]:
x = np.zeros((len(prev_words),word_length, len(unique_word)), dtype=bool)
y = np.zeros((len(next_words),len(unique_word)),dtype=bool)
for i, each_words in enumerate(prev_words):
    for j, each_words in enumerate(each_words):
        x[i,j, unique_word_index[each_words]] = 1
    y[i,unique_word_index[next_words[i]]] =1

In [11]:
print(x[0][0])

[False False False ... False False False]
[False False False ... False False False]


# Building the model

In [13]:
model = Sequential()
model.add(LSTM(128,input_shape=(word_length, len(unique_word))))
model.add(Dense(len(unique_word)))
model.add(Activation('softmax'))

# Training our Model 

In [15]:
optimizer = RMSprop(lr=0.01)
model.compile(loss='categorical_crossentropy',optimizer=optimizer,metrics=['accuracy'])
history = model.fit(x,y,validation_split=0.05,batch_size=128,epochs=2,shuffle=True).history

C:\ProgramData\Anaconda3\lib\site-packages\keras\optimizers\optimizer_v2\rmsprop.py:140: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super().__init__(name, **kwargs)
C:\ProgramData\Anaconda3\lib\site-packages\keras\optimizers\optimizer_v2\rmsprop.py:140: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super().__init__(name, **kwargs)


Epoch 1/2
Epoch 1/2
811/811 [==============================] - 309s 374ms/step - loss: 5.9998 - accuracy: 0.1083 - val_loss: 7.0276 - val_accuracy: 0.1020
Epoch 2/2
Epoch 2/2
811/811 [==============================] - 238s 293ms/step - loss: 5.7671 - accuracy: 0.1474 - val_loss: 7.8632 - val_accuracy: 0.1060


# Loading our Model

In [17]:
model.save('keras_next_word_model.h5')
pickle.dump(history, open('history.p','wb'))
model = load_model('keras_next_word_model.h5')
history = pickle.load(open('history.p','rb'))

# Making Predictions

In [19]:
def prepare_input(text):
    x = np.zeros((1,word_length, len(unique_word)))
    for t, word in enumerate(text.split()):
        print(word)
        x[0, t,unique_word_index[word]] = 1
    return x
prepare_input("How are you ".lower())

how
are
you
how
are
you


array([[[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]]])

array([[[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]]])

In [20]:
def sample(preds, top_n=3):
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds)
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    return heapq.nlargest(top_n, range(len(preds)),preds.take)

# Function for prediction

In [25]:
def predict_completion(text, n=3):
    if text == "":
        return("0")
    x = prepare_input(text)
    preds = model.predict(x, verbose=0)[0]
    next_indices = sample(preds, n)
    return [unique_word[idx] for idx in next_indices]

# Result

In [26]:
q = "Tell me about your self and your family"
print("correct sentence: ",q)
seq = " ".join(tokenizer.tokenize(q.lower())[0:5])
print("Sequence: ",seq)
print("Next Posible words are: ",predict_completion(seq, 5))

correct sentence:  Tell me about your self and your family
Sequence:  tell me about your self
tell
me
about
your
self
correct sentence:  Tell me about your self and your family
Sequence:  tell me about your self
tell
me
about
your
self
Next Posible words are:  ['s', 'is', 'you', 'i', 'for']
Next Posible words are:  ['s', 'is', 'you', 'i', 'for']


# Thank You.... :)